# Build Data Duration Hedge CRSP

### CRSP Treasury Manual

https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Treasury_Database_Guide_for_SAS_ASCII_EXCEL_R.pdf

https://www.crsp.org/products/documentation/treasno-properties

# WRDS API

In [1]:
import wrds
file_key = open("../../keys/wrds_username.txt","r")
USERNAME = file_key.read()
file_key.close()

db = wrds.Connection(wrds_username=USERNAME)
#db.create_pgpass_file()

Loading library list...
Done


In [2]:
import pandas as pd
import numpy as np
from datetime import date

In [3]:
DATE = '2024-05-31'

# Data API

In [6]:
df = data.copy()
df = df[~df['itype'].isin([11,12])]
df.dropna(subset=['tdduratn'],inplace=True)

NameError: name 'data' is not defined

In [7]:
PERIODS_GRID = 21
FREQ = '6M'

In [8]:
from pandas.tseries.offsets import DateOffset

# Assuming DATE and data are given
# example dataframe 'data' with 'tmatdt' and 'tdatdt' columns

# Convert DATE and columns in 'data' to datetime
dateobj = pd.to_datetime(DATE)
df['tmatdt'] = pd.to_datetime(df['tmatdt'])
df['tdatdt'] = pd.to_datetime(df['tdatdt'])

# Generate 6-month intervals from DATE
six_month_intervals = pd.date_range(start=dateobj, periods=PERIODS_GRID, freq=FREQ) # adjust periods as needed

# Function to find closest date in 'tmatdt' for each interval
def find_closest_date(interval, data):
    # Calculate the absolute difference between each MATURITY date and the interval
    data['difference'] = abs(data['tmatdt'] - interval)
    # Ensure we only consider future dates relative to DATE
    future_dates = data[data['tmatdt'] > dateobj]
    if not future_dates.empty:
        # Find the row with the minimum difference
        min_diff = future_dates['difference'].min()
        closest_dates = future_dates[future_dates['difference'] == min_diff]
        # Resolve ties by 'tdatdt' date
        return closest_dates.sort_values('tdatdt', ascending=False).iloc[0]
    return None

# Apply the function to each interval
selected_rows = [find_closest_date(interval, df) for interval in six_month_intervals]

# Remove None values and ensure uniqueness
selected_rows = [row for row in selected_rows if row is not None]
data_select = pd.DataFrame(selected_rows).drop_duplicates(subset='tmatdt')

# Add new column for years difference
data_select['TTM'] = data_select['tmatdt'].apply(
    lambda x: (x - dateobj).days / 365.25)

data_select = data_select[['kytreasno','caldt','tdatdt','tmatdt','TTM','tdbid','tdask','tdaccint','tdyld','tdduratn','tdtotout','tcouprt','itype']]

data_select['kytreasno'] = data_select['kytreasno'].map('{:.0f}'.format)
data_select = data_select.set_index('kytreasno')

NameError: name 'df' is not defined

In [9]:
selected = data_select.rename(columns={'caldt':'date','tdatdt':'issue date','tdtotout':'outstanding','tmatdt':'maturity date','TTM':'ttm','tdbid':'bid','tdask':'ask','tcouprt':'cpn rate','itype':'instrument','tdyld':'ytm','tdaccint':'accrint', 'tdduratn':'duration'})

# eliminate issue 0 years away
selected = selected[~(selected['duration']<30)]

selected['ytm'] *= 365.25
selected['duration'] /= 365.25

selected.style.format({
    'ttm':'{:.2f}','tdbid':'{:.2f}',
    'ask':'{:.2f}',
    'accrint':'{:.2f}',
    'ytm':'{:.2f}',
    'duration':'{:.2f}',
    'outstanding':'{:,.0f}',
    'cpn rate':'{:.3f}',
    'instrument':'{:.0f}',
    'issue date': lambda x: x.strftime('%Y-%m-%d') if not pd.isnull(x) else '',
    'maturity date': lambda x: x.strftime('%Y-%m-%d') if not pd.isnull(x) else ''
})

NameError: name 'data_select' is not defined

In [10]:
selected['price'] = (selected['bid'] + selected['ask'])/2 + selected['accrint']

NameError: name 'selected' is not defined

In [11]:
# with pd.option_context('display.max_rows',None):
#     display(data_select.sort_values('tdatdt')[['kytreasno','tmatdt','tdatdt','tcouprt']])

In [12]:
KEYNOS = (207392, 207391)

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_dly where kytreasno in {KEYNOS}'
data_quotes = db.raw_sql(SQL_QUERY)

SQL_QUERY = f'select * from crsp_m_treasuries.tfz_iss where kytreasno in {KEYNOS}'
data_iss = db.raw_sql(SQL_QUERY)

rawdata = data_quotes.merge(data_iss, on='kytreasno')

***

# Process Data

In [13]:
import sys
sys.path.insert(0, '../cmds')
from treasury_cmds import process_wrds_treasury_data

data = process_wrds_treasury_data(rawdata,keys_extra=['duration'])

***

# Get Timeseries for Duration Analysis

Update the KEYNOs. 
* These three are issued in Aug 2019.
* Idea is 5, 10, 30 maturities.
* 5-year matures in Aug 2024. So will need updated.

Used in HW 2, problem 3.
* Only make use of the 30yr and 10yr there.

In [14]:
px = data.pivot_table(index='quote date',columns='KYTREASNO',values='price').dropna()
duration = data.pivot_table(index='quote date',columns='KYTREASNO',values='duration').dropna()

***

# Save to Excel

In [15]:
outfile = f'../data/treasury_ts_duration_{DATE}.xlsx'
with pd.ExcelWriter(outfile) as writer:      
    data.to_excel(writer, sheet_name= 'database', index=True)
    px.to_excel(writer, sheet_name= 'price', index=True)
    duration.to_excel(writer, sheet_name= 'duration', index=True)